In [ ]:
import pdfplumber
import google.generativeai as genai
import typing_extensions as typing
import json



In [ ]:
def extract_text_from_pdf(pdf_file_path: str) -> str:
    with pdfplumber.open(pdf_file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text() + "\n"
        return text

In [65]:
path = "pdfs/BBT-Bank-Statement-EVERGREEN 6.17.2020-1-1.pdf"

text = extract_text_from_pdf(path)
with open("output.txt", "w") as file:
    file.write(extract_text_from_pdf(path))

In [10]:
model = genai.GenerativeModel("gemini-1.5-flash")

In [68]:
text

'Page 1 of 5 06/17/20\nDC 1090001004290\n999-99-99-99 16769 3 C 001 11 S 66 002\nEVERGREEN SUPPLIERS\n88 SUNRISE BLVD\nRIVERSIDE, CA 92506\nYour consolidated statement\nContact us\n(600) BANK-BBT or\nFor 06/17/2020 BBT.com\n(800) 226-5228\nDo more with digital banking\nBank without having to leave home. Check your account balances, make transfers, pay bills and deposit checks with your mobile device. If\nyou are not enrolled in digital banking, it only lakes a minute Gel started today at BBT.com/U.\nTruist Bank, Member FDIC. To learn more, visit BBT.com/Truist. ©2020 Truist Financial Corporation.\nIf you are traveling outside of the USA and have concerns about accessing your account while you are traveling, please contact your\nBranch Banker or call us at 800-BANK BBT.\nSummary of your accounts\nACCOUNT NAME ACCOUNT NUMBER BALANCE ($) DETAILS ON\nBRIGHT BANKING 1090001004290 964.80 page 1\nTotal checking and money market savings accounts $964.80\nBRIGHT BANKING 1090001004290 7.60 page 

In [69]:
prompt = """The following text is a bank statement containing structured tables of transaction data along with other text. Extract all the tables from the text. For each table:
1. Assign a descriptive title based on the content of the table (e.g., "Account Summary", "Checks", "Deposits and Credits").
2. Format the table as JSON with the following structure:
   {
       "title": "Table Title",
       "header": ["Column1", "Column2", "Column3"],
       "rows": [
           ["Row1Value1", "Row1Value2", "Row1Value3"],
           ["Row2Value1", "Row2Value2", "Row2Value3"]
       ]
   }
3. If multiple tables exist, return them as a list of JSON objects under the key `"tables"`. Ensure each table has a unique title.
   
Text data from the bank statement:"""

full_prompt = f"{prompt}\n\n{text}"
full_prompt

'The following text is a bank statement containing structured tables of transaction data along with other text. Extract all the tables from the text. For each table:\n1. Assign a descriptive title based on the content of the table (e.g., "Account Summary", "Checks", "Deposits and Credits").\n2. Format the table as JSON with the following structure:\n   {\n       "title": "Table Title",\n       "header": ["Column1", "Column2", "Column3"],\n       "rows": [\n           ["Row1Value1", "Row1Value2", "Row1Value3"],\n           ["Row2Value1", "Row2Value2", "Row2Value3"]\n       ]\n   }\n3. If multiple tables exist, return them as a list of JSON objects under the key `"tables"`. Ensure each table has a unique title.\n   \nText data from the bank statement:\n\nPage 1 of 5 06/17/20\nDC 1090001004290\n999-99-99-99 16769 3 C 001 11 S 66 002\nEVERGREEN SUPPLIERS\n88 SUNRISE BLVD\nRIVERSIDE, CA 92506\nYour consolidated statement\nContact us\n(600) BANK-BBT or\nFor 06/17/2020 BBT.com\n(800) 226-5228

In [43]:
class ExtractedData(typing.TypedDict):
    date: str
    name_of_transaction: str
    amount: float

In [70]:
result = model.generate_content(
    full_prompt,
    generation_config=genai.GenerationConfig(
        response_mime_type="application/json"
    )
)
print(result)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "{\"tables\": [{\"title\": \"Account Summary\", \"header\": [\"Previous Balance as of 05/18/2020\", \"Checks\", \"Other Withdrawals, Debits and Service Charges\", \"Deposits, Credits and Interest\", \"New Balance as of 06/17/2020\", \"Average Posted Balance in Statement Cycle\"], \"rows\": [[\"$2,074.21\", \"-701.39\", \"-6,242.34\", \"+5,834.32\", \"$964.80\", \"$1,048.00\"]]}, {\"title\": \"Checks\", \"header\": [\"Date\", \"Check #\", \"Amount ($)\"], \"rows\": [[\"05/26\", \"1401\", \"450.00\"], [\"06/05\", \"*965025\", \"101.39\"], [\"06/09\", \"985026\", \"150.00\"]]}, {\"title\": \"Other Withdrawals, Debits and Service Charges\", \"header\": [\"Date\", \"Description\", \"Amount ($)\"], \"rows\": [[\"05/19\", \"BB&T ONLINE BILL PAYMENT ONLINE PMT CREDIT

In [71]:
text_result = json.loads(result.candidates[0].content.parts[0].text)
print(text_result)

{'tables': [{'title': 'Account Summary', 'header': ['Previous Balance as of 05/18/2020', 'Checks', 'Other Withdrawals, Debits and Service Charges', 'Deposits, Credits and Interest', 'New Balance as of 06/17/2020', 'Average Posted Balance in Statement Cycle'], 'rows': [['$2,074.21', '-701.39', '-6,242.34', '+5,834.32', '$964.80', '$1,048.00']]}, {'title': 'Checks', 'header': ['Date', 'Check #', 'Amount ($)'], 'rows': [['05/26', '1401', '450.00'], ['06/05', '*965025', '101.39'], ['06/09', '985026', '150.00']]}, {'title': 'Other Withdrawals, Debits and Service Charges', 'header': ['Date', 'Description', 'Amount ($)'], 'rows': [['05/19', 'BB&T ONLINE BILL PAYMENT ONLINE PMT CREDIT CARD CKF768752662POS', '100.00'], ['05/19', 'BB&T ONLINE BILL PAYMENT ONLINE PMT FIRST PROGRESS CKF768752662POS', '100.00'], ['05/19', 'BB&T ONLINE BILL PAYMENT ONLINE PMT MERRICK BANK CKF768752662POS', '100.00'], ['05/19', 'BB&T ONLINE BILL PAYMENT ONLINE PMT CREDIT ONE BANK CKF768752662POS', '200.00'], ['05/19'

In [67]:
text_result

{'tables': [{'header': ['DATE', 'CHECK #', 'AMOUNT ($)'],
   'rows': [['04/26', '1401', '450.00'],
    ['05/05', '*965025', '101.39'],
    ['05/09', '985026', '150.00']]},
  {'header': ['DATE', 'DESCRIPTION', 'AMOUNT ($)'],
   'rows': [['04/19',
     'BB&T ONLINE BILL PAYMENT ONLINE PMT CREDIT CARD CKF768752662POS',
     '100.00'],
    ['04/19',
     'BB&T ONLINE BILL PAYMENT ONLINE PMT FIRST PROGRESS CKF768752662POS',
     '100.00'],
    ['04/19',
     'BB&T ONLINE BILL PAYMENT ONLINE PMT MERRICK BANK CKF768752662POS',
     '100.00'],
    ['04/19',
     'BB&T ONLINE BILL PAYMENT ONLINE PMT CREDIT ONE BANK CKF768752662POS',
     '200.00'],
    ['04/19',
     'DEBIT CARD PURCHASE-PIN 05-18-20 WASHINGTON DC 6901 SAFEWAY 2808',
     '27.21'],
    ['04/19', 'CC PMT LEADBANKSELFLEND 6438 RICARDO BROUGHTON', '62.62'],
    ['04/20', 'INTERNET PAYMENT PAYMENT FSB BLAZE 518213010852770', '75.00'],
    ['04/21',
     'DEBIT CARD PURCHASE SHELL OIL 10000581 05-19 WASHINGTON DC 6901',
     '1.99']